# Smart Conversational Agent for Mental Health Support

## Design Approach: Generative-Based

### Overview

This project aims to create a smart conversational agent designed to provide mental health support. The agent utilizes a generative-based approach, enabling it to offer empathetic conversations, coping strategies, and resources for users experiencing stress, anxiety, or depression. Additionally, the agent can guide users to professional help if needed.

### Table of Contents

1. [Introduction](#introduction)
2. [Objectives](#objectives)
3. [Technologies Used](#technologies-used)
4. [Dataset and Preprocessing](#dataset-and-preprocessing)
5. [Generative-Based Design](#generative-based-design)
6. [Implementation](#implementation)
    1. [GPT2 OpenAi](#GPT2-OpenAi)
    2. [Cohere Ai](#Cohere-Ai)
    3. [MistralAi](#MistralAi)
    4. [Ai21 Studio](#Ai21-Studio)
    5. [GPT 3.5-Turbo](#GPT-3.5-Turbo)
    6. [Generating Responses](#generating-responses)
7. [User Interaction](#user-interaction)
8. [Examples](#examples)
9. [Future Enhancements](#future-enhancements)
10. [Conclusion](#conclusion)

### Introduction

Mental health is a crucial aspect of overall well-being, and many people experience stress, anxiety, or depression at some point in their lives. While professional help is essential, immediate support and coping strategies can be beneficial. This project aims to bridge that gap by providing a conversational agent capable of offering support and guidance.

### Objectives

- To provide empathetic conversation to users experiencing mental health issues.
- To offer coping strategies and resources.
- To guide users to professional help when necessary.

### Technologies Used

- Python
- Natural Language Processing (NLP)
- Transformers (Hugging Face)
- PyTorch
- Pandas
- TensorFlow

### Dataset and Preprocessing

The dataset used for training the conversational agent consists of questions and answers related to mental health support. Preprocessing steps include text normalization, tokenization, and padding.

### Generative-Based Design

The agent employs a generative-based design using a pre-trained language model (GPT-2) fine-tuned on a dataset of mental health conversations. This design allows the agent to generate natural and contextually appropriate responses to user inputs.

### Implementation

In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader

df = pd.read_csv('Dataset/mentalhealthc.csv')
#65 Input Questions and Answers
Questions = df['Questions']
Answers = df['Answers']

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW

# Load the tokenizer and add padding token
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Verify that the padding token has been set correctly
print(tokenizer.pad_token, tokenizer.eos_token)

<|endoftext|> <|endoftext|>


In [3]:
class ConversationalDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context = self.data['Questions'].iloc[idx]
        response = self.data['Answers'].iloc[idx]
        encoding = self.tokenizer(context + self.tokenizer.eos_token + response, 
                                  truncation=True, padding='max_length', 
                                  max_length=self.max_length, return_tensors="pt")
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        labels = encoding['input_ids'].squeeze()
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

dataset = ConversationalDataset(df, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [4]:
import torch

model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Resize the embeddings to accommodate the new pad token
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)

C:\Users\user\anaconda3\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [5]:
def train(model, dataloader, optimizer, device, epochs=3):
    model.train()
    for epoch in range(epochs):
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            optimizer.zero_grad()
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            
            print(f"Epoch: {epoch}, Loss: {loss.item()}")

train(model, dataloader, optimizer, device)

Epoch: 0, Loss: 7.6629533767700195
Epoch: 0, Loss: 6.399724960327148
Epoch: 0, Loss: 6.783846378326416
Epoch: 0, Loss: 4.640393257141113
Epoch: 0, Loss: 4.726998805999756
Epoch: 0, Loss: 4.434204578399658
Epoch: 0, Loss: 3.984682083129883
Epoch: 0, Loss: 3.070922613143921
Epoch: 1, Loss: 2.8975698947906494
Epoch: 1, Loss: 2.310420274734497
Epoch: 1, Loss: 2.5816383361816406
Epoch: 1, Loss: 2.0184409618377686
Epoch: 1, Loss: 1.842942714691162
Epoch: 1, Loss: 2.766465425491333
Epoch: 1, Loss: 1.8473297357559204
Epoch: 1, Loss: 2.0855367183685303
Epoch: 2, Loss: 2.224907159805298
Epoch: 2, Loss: 1.9629104137420654
Epoch: 2, Loss: 1.4977854490280151
Epoch: 2, Loss: 1.653773307800293
Epoch: 2, Loss: 1.495230793952942
Epoch: 2, Loss: 1.009582281112671
Epoch: 2, Loss: 1.4870004653930664
Epoch: 2, Loss: 0.7564858198165894


In [ ]:
def generate_response(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens= True)

# Interactive chat loop
print("Chat with the model (type 'quit' to stop)")

while True:
    user_input = input("User: ")
    if user_input.lower() == 'quit':
        break
    response = generate_response(user_input)
    print(f"Bot: {response}")

## Cohere

In [31]:
import cohere
import pandas as pd

# Load the dataset
df = pd.read_csv('Dataset/mentalhealthc.csv')
questions = df['Questions'].tolist()
answers = df['Answers'].tolist()

# Initialize the Cohere client with your API key
co = cohere.Client('YOUR_APIKEY')

In [32]:
def generate_prompt(user_input):
    prompt = "You are a mental health assistant. Answer the following questions based on the provided context:\n\n"
    for question, answer in zip(questions, answers):
        prompt += f"Q: {question}\nA: {answer}\n\n"
    prompt += f"Q: {user_input}\nA: "
    return prompt
def chat_with_cohere(user_input):
    prompt = generate_prompt(user_input)
    response = co.generate(
        model='command-xlarge-nightly',
        prompt=prompt,
        max_tokens=500
    )
    return response.generations[0].text.strip()

In [33]:
def main():
    print("Cohere Chatbot: Type 'exit' to end the chat.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Cohere Chatbot: Goodbye!")
            break
        
        response = chat_with_cohere(user_input)
        print(f"Cohere Chatbot: {response}")

if __name__ == "__main__":
    main()

Cohere Chatbot: Type 'exit' to end the chat.


You:  Hi, i need assistance


Cohere Chatbot: I'm here to help! How can I assist you today?


You:  I want to ask about mental illness?


Cohere Chatbot: Mental illness refers to health conditions that disrupt a person's thoughts, emotions, relationships, and daily functioning. Mental illness can affect anyone, regardless of age, gender, ethnicity, or background. Symptoms vary depending on the specific condition and can range from mild to severe. Early identification and treatment are crucial for recovery. If you or someone you know is exhibiting signs of mental illness, it is important to talk to a trusted friend or family member and seek professional help from a mental health specialist.


You:  What should we do if someone want suicide?


Cohere Chatbot: If someone you know is talking about suicide, it's important to take them seriously. Here are some steps you can take:

1. Ask open-ended questions to encourage them to talk and share their feelings. This can help you understand their reasons and thoughts behind their suicidal thoughts.
2. Listen non-judgmentally and reflect on what they are saying to show that you hear and understand their perspective.
3. Show that you care and want to help. Let them know that you are concerned and want to support them.
4. Encourage professional help and offer to assist them in finding a mental health professional or hotlines they can call.
5. Remove any means they might have to harm themselves, such as weapons or large amounts of medication.
6. Stay with them or make sure they are in a safe place with people who can look after them.
7. Be supportive and keep in touch. Let them know that you are there for them and check in with them regularly.

Remember, it's important to take any talk

You:  exit


Cohere Chatbot: Goodbye!


## MistralAi

In [1]:
import pandas as pd
import requests
import json

api_key = 'YOUR_APIKEY'

def call_chat_endpoint(data, api_key=api_key):
    url = "https://codestral.mistral.ai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Load the dataset
df = pd.read_csv('Dataset/mentalhealthc.csv')

# Convert the dataset to a format suitable for context
context = ""
for index, row in df.iterrows():
    context += f"Q: {row['Questions']}\nA: {row['Answers']}\n"

def generate_response(user_input, context):
    prompt = f"{context}\nQ: {user_input}\nA:"

    data = {
        "model": "codestral-latest",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7
    }

    response = call_chat_endpoint(data)
    if isinstance(response, dict) and 'choices' in response:
        return response['choices'][0]['message']['content']
    else:
        return response

# Simulate user interaction
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        break
    response = generate_response(user_input, context)
    print(f"Bot: {response}")

You:  Hi, How're you?


Bot: I'm an assistant and I'm here to provide information and support based on the data I've been trained on. I don't have personal experiences, and I'm unable to provide medical advice or diagnose any conditions. I'm here to help answer your questions about mental health, addiction, and related topics. How can I assist you today?


You:  I need assistance regarding my mental health, am i talking to mental health specialist?


Bot: Yes, you are. I'm here to help you understand more about mental health, provide information, and offer guidance. If you're looking for professional help or advice, it's always best to consult with a licensed mental health professional. They can provide personalized support and treatment based on your unique circumstances.


You:  What about you?


...

You:  If something is causing disturbance in our heads, how can we change our mind and feel relaxed.


Bot: Changing one's mindset and promoting relaxation can be achieved through various methods. Here are a few suggestions:

1. **Mindfulness Meditation**: This practice involves focusing your awareness on the present moment, without judgment. This can help calm the mind and promote relaxation.

2. **Progressive Muscle Relaxation**: This technique involves tensing and then releasing each muscle group in the body. This can help reduce anxiety and promote relaxation.

3. **Deep Breathing Exercises**: Slow, deep breathing can help lower blood pressure, relax muscles, and reduce anxiety.

4. **Yoga and Stretching**: These practices can help reduce tension, improve flexibility, and promote relaxation.

5. **Listening to Calming Music**: Music can have a significant impact on mood and relaxation. Listening to soothing music can help calm the mind and promote relaxation.

6. **Reading or Listening to Relaxing Content**: Engaging in activities that involve reading or listening to calming content

You:  Thank you So much or your exisitance, i'm glad i'd you.


Bot: You're very welcome! I'm here to help. If you have any more questions or concerns, feel free to ask. I'm here to provide information and support in a friendly and helpful manner.


You:  exit


## AI21 Studio

In [6]:
import pandas as pd
import requests
import json

api_key = 'YOUR_APIKEY'

def call_ai21_endpoint(prompt, api_key=api_key):
    url = "https://api.ai21.com/studio/v1/j2-large/complete"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    data = {
        "prompt": prompt,
        "numResults": 1,
        "maxTokens": 200,
        "stopSequences": ["\n"],
        "topKReturn": 0,
        "temperature": 0.7
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Load the dataset
df = pd.read_csv('Dataset/mentalhealthc.csv')

# Convert the dataset to a format suitable for context
context = ""
for index, row in df.iterrows():
    context += f"Q: {row['Questions']}\nA: {row['Answers']}\n"

def truncate_context(context, max_tokens=8000):
    words = context.split()
    truncated_context = ''
    token_count = 0

    for word in words:
        token_count += len(word) + 1  # Adding 1 for the space
        if token_count > max_tokens:
            break
        truncated_context += word + ' '
    
    return truncated_context.strip()

def generate_response(user_input, context, max_tokens=8000):
    truncated_context = truncate_context(context, max_tokens)
    prompt = f"{truncated_context}\nQ: {user_input}\nA:"

    response = call_ai21_endpoint(prompt)
    if isinstance(response, dict) and 'completions' in response:
        return response['completions'][0]['data']['text'].strip()
    else:
        return response

# Simulate user interaction
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        break
    response = generate_response(user_input, context)
    print(f"Bot: {response}")

You:  Hi, how are you?


Bot: I'm good, thanks! I'm learning a lot about mental health, and I'd love to continue the conversation.


You:   I need assistance regarding my mental health, am i talking to mental health specialist?


Bot: While it's true that mental health specialists can help, it's important to understand that there are many other mental health professionals who can provide assistance. Psychologists, social workers, mental health nurses, counsellors, and psychiatrists are all examples of mental health professionals who can provide guidance and support. It's important to find a mental health professional who you feel most comfortable working with.


You:  What about you?


...

You:  I'm a data scientist and i made you.


Bot: Thank you for making me. I enjoy working with data.


You:  Thank you for your assistance Exit


Bot: We appreciate the opportunity to provide you with assistance.


You:  exit


In [8]:
import cohere
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load the dataset
df = pd.read_csv('Dataset/mentalhealthc.csv')
questions = df['Questions'].tolist()
answers = df['Answers'].tolist()

# Initialize the Cohere client with your API key
co = cohere.Client('YOUR_APIKEY')

# Email alert configuration
def send_email_alert(user_input):
    sender_email = "ahmadmakki5542@gmail.com"
    receiver_email = "makkia52@gmail.com"
    password = "makkimakki"

    subject = "Alert: Harmful Language Detected"
    body = f"The following harmful language was detected in the chat:\n\n{user_input}"

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, password)
            text = msg.as_string()
            server.sendmail(sender_email, receiver_email, text)
        print("Email alert sent successfully.")
    except Exception as e:
        print(f"Failed to send email alert: {e}")

def generate_prompt(user_input):
    prompt = "You are a mental health assistant. Answer the following questions based on the provided context:\n\n"
    for question, answer in zip(questions, answers):
        prompt += f"Q: {question}\nA: {answer}\n\n"
    prompt += f"Q: {user_input}\nA: "
    return prompt

def chat_with_cohere(user_input):
    harmful_words = ["suicide", "kill myself", "self-harm", "end my life", "harm myself",
    "take my life", "commit suicide", "die by suicide", "attempt suicide", "self-injury",
    "self-mutilation", "hurt myself", "cut myself", "burn myself", "jump off a bridge",
    "overdose", "hang myself", "shoot myself", "stab myself", "slit my wrists",
    "poison myself", "drown myself", "electrocute myself", "suffocate myself", "asphyxiate myself",
    "lethal dose", "carbon monoxide", "helium hood", "jump in front of a train", "jump in front of a car",
    "starve myself", "refuse to eat", "fast until death", "die from exposure", "freeze to death",
    "set myself on fire", "self-immolation", "jump from a building", "fall to my death", "crash my car intentionally",
    "crash my plane intentionally", "take sleeping pills", "overdose on drugs", "take too many pills", "cut my throat",
    "cut my jugular", "bleed out", "exsanguinate", "take poison", "drink bleach",
    "drink antifreeze", "drink drain cleaner", "inject air into veins", "inject insulin", "inject potassium chloride",
    "swallow glass", "swallow razor blades", "swallow batteries", "jump into traffic", "jump into water with weights",
    "jump off a cliff", "throw myself off", "drive off a cliff", "drive into a wall", "walk into traffic",
    "step in front of a bus", "step in front of a train", "walk onto train tracks", "walk onto highway", "walk into ocean",
    "swim out to sea", "stay underwater", "stay in cold water", "freeze myself", "lock myself in a freezer",
    "hold my breath", "stop breathing", "put a plastic bag over my head", "tape a bag around my head", "inhale nitrogen",
    "inhale helium", "inhale argon", "inhale carbon dioxide", "inhale car exhaust", "inhale fumes",
    "jump from height", "fall from height", "intentional overdose", "fatal overdose", "lethal injection",
    "drink pesticide", "drink insecticide", "ingest toxin", "ingest poison", "lethal self-inflicted injury",
    "fatal self-inflicted injury", "intentional self-destruction", "deliberate self-destruction", "kill myself intentionally"]

    if any(word in user_input.lower() for word in harmful_words):
        send_email_alert(user_input)
    
    prompt = generate_prompt(user_input)
    response = co.generate(
        model='command-xlarge-nightly',
        prompt=prompt,
        max_tokens=50
    )
    return response.generations[0].text.strip()

def main():
    print("Cohere Chatbot: Type 'exit' to end the chat.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Cohere Chatbot: Goodbye!")
            break
        
        response = chat_with_cohere(user_input)
        print(f"Cohere Chatbot: {response}")

if __name__ == "__main__":
    main()

Cohere Chatbot: Type 'exit' to end the chat.


You:  hi


Cohere Chatbot: Hello, how can I help you today?


You:  exit


Cohere Chatbot: Goodbye!


In [6]:
pip install --upgrade httpx openai

Note: you may need to restart the kernel to use updated packages.


In [8]:
import openai
import pandas as pd
from packaging import version

required_version = version.parse("1.1.1")
current_version = version.parse(openai.__version__)

if current_version < required_version:
    raise ValueError(f"Error: OpenAI version {openai.__version__}"
                     " is less than the required version 1.1.1")
else:
    print("OpenAI version is compatible.")

# Set up OpenAI API key
openai.api_key = "YOUR_APIKEY"

# Load the provided mental health data
file_path = 'mentalhealthc.csv'
mental_health_data = pd.read_csv(file_path)

# Function to retrieve a factual response from the dataset
def get_factual_response(user_query):
    for _, row in mental_health_data.iterrows():
        if row['Questions'].lower() in user_query.lower():
            return row['Answers']
    return None

# Function to generate a response using OpenAI API
def generate_response(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a compassionate mental health support agent."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message['content'].strip()

# Main function to handle user input and provide responses
def mental_health_agent(user_input):
    # Try to get a factual response from the dataset
    factual_response = get_factual_response(user_input)
    
    if factual_response:
        return factual_response
    else:
        # Define a base prompt for the language model
        base_prompt = f"User: {user_input}\nAgent:"
        
        # Generate a response using the OpenAI API
        response = generate_response(base_prompt)
        
        return response

# Continuous loop to take user input
def main():
    print("Welcome to the Mental Health Support Agent. Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Agent: Thank you for using the Mental Health Support Agent. Take care!")
            break
        response = mental_health_agent(user_input)
        print(f"Agent: {response}")

if __name__ == "__main__":
    main()

ImportError: cannot import name 'BaseTransport' from 'httpx' (C:\Users\user\anaconda3\Lib\site-packages\httpx\__init__.py)

In [36]:
import cohere
import pandas as pd
from langchain.chains import SimpleChain
from langchain.prompts import PromptTemplate
from langchain.llms import Cohere as LangCohere

# Load the dataset
df = pd.read_csv('Dataset/mentalhealthc.csv')
questions = df['Questions'].tolist()
answers = df['Answers'].tolist()

# Initialize the Cohere client with your API key
cohere_api_key = 'knU6ocEmAGAYqKpj0lnz7Ov9qs1zJvIpuLqNPpb9'
co = cohere.Client(cohere_api_key)

# Create a LangChain LLM instance
lang_cohere = LangCohere(api_key=cohere_api_key, model='command-xlarge-nightly')

# Function to generate a prompt for LangChain
def generate_langchain_prompt(user_input):
    context = "You are a mental health assistant. Answer the following questions based on the provided context:\n\n"
    for question, answer in zip(questions, answers):
        context += f"Q: {question}\nA: {answer}\n\n"
    context += f"Q: {user_input}\nA: "
    return context

# LangChain prompt template
prompt_template = PromptTemplate(input_variables=["user_input"], template="{context}\nQ: {user_input}\nA:")

# Function to create a LangChain prompt and get a response
def chat_with_langchain(user_input):
    context = generate_langchain_prompt(user_input)
    prompt = prompt_template.render(context=context, user_input=user_input)
    response = lang_cohere(prompt=prompt, max_tokens=50)
    return response["choices"][0]["text"].strip()

def main():
    print("Cohere Chatbot: Type 'exit' to end the chat.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Cohere Chatbot: Goodbye!")
            break
        
        response = chat_with_langchain(user_input)
        print(f"Cohere Chatbot: {response}")

if __name__ == "__main__":
    main()

ImportError: cannot import name 'SimpleChain' from 'langchain.chains' (C:\Users\user\anaconda3\Lib\site-packages\langchain\chains\__init__.py)

In [35]:
pip install langchain --upgrade

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Check cohere library installation
!pip show cohere

# Import cohere
import cohere

# Initialize the Cohere client
co = cohere.Client('knU6ocEmAGAYqKpj0lnz7Ov9qs1zJvIpuLqNPpb9')  # Ensure your API key is correctly inserted here

# Quick check to see if the client is set up
print(co)

Name: cohere
Version: 5.1.5
Summary: 
Home-page: 
Author: 
Author-email: 
License: 
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: httpx, pydantic, typing_extensions
Required-by: 


In [ ]:
response = co.generate(
    model='large',
    prompt='Hello, world!',
    max_tokens=5
)
print(response.generations[0].text)

BadRequestError: status_code: 400, body: {'message': "invalid request: model 'large' is not supported by the generate API"}

In [ ]:
import cohere
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# Load the dataset
df = pd.read_csv('Dataset/mentalhealthc.csv')

# Cohere API Key
co = cohere.Client('knU6ocEmAGAYqKpj0lnz7Ov9qs1zJvIpuLqNPpb9')

# Define the dataset
class ConversationalDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context = self.data['Questions'].iloc[idx]
        response = self.data['Answers'].iloc[idx]
        return {"prompt": context + "[END]", "completion": response}

# Create dataset
dataset = ConversationalDataset(df)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Prepare data for training
training_examples = []
for data in dataloader:
    training_examples.append(data)

# Fine-tune the model
response = co.fine_tune.create(
  model="large",
  dataset=training_examples,
  epochs=3,
  learning_rate_multiplier=0.1,
  prompt_loss_weight=1.0
)

print(f"Fine-tuning started: {response['id']}")

# Check the status of the fine-tuning job
status = co.fine_tune.retrieve(response['id'])
print(f"Fine-tuning status: {status['status']}")

AttributeError: 'Client' object has no attribute 'fine_tune'

In [4]:
import pandas as pd
import re

# Load the dataset
df = pd.read_csv('Dataset/Mental_Health_Conversational_Data_set.csv')

# Display the first few rows of the dataframe
print("Original Data:")
print(df.head())

# Data Cleaning
# Remove any rows that have missing values
df.dropna(inplace=True)

# Removing any duplicate rows, keeping the first occurrence
df.drop_duplicates(inplace=True)

# Clean the text: removing extra spaces, special characters
def clean_text(text):
    # Convert to lower case
    text = text.lower()
    # Remove unwanted characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['Context'] = df['Context'].apply(clean_text)
df['Response'] = df['Response'].apply(clean_text)

# Data Annotation (Example of adding a simple annotation for sentiment, replace with relevant tagging)
# For illustration purposes only, as proper sentiment analysis or intent classification should use a trained model or deeper analysis
def annotate_data(response):
    support_keywords = [
        'sad', 'depressed', 'anxious', 'help', 'stress', 'hurt', 'disrespected', 
        'therapy', 'counseling', 'advice', 'feeling down', 'worried', 'panic', 'support',
        'mental health', 'cope', 'coping', 'struggle', 'struggling', 'overwhelmed',
        'distressed', 'problem', 'issue', 'difficulty', 'frustration', 'concern',
        'self-care', 'affirmations', 'crisis', 'emotional', 'feeling low', 'isolation'
    ]
    if any(word in response for word in support_keywords):
        return 'Support'
    else:
        return 'General'

df['Annotation'] = df['Response'].apply(annotate_data)

# Display the cleaned and annotated data
print("\nCleaned and Annotated Data:")
print(df.head())

# Save the cleaned and annotated data to a new CSV file
df.to_csv('Cleaned_Mental_Health_Conversational_Data.csv', index=False)

Original Data:
                                             Context  \
0  I'm going through some things with my feelings...   
1  I'm going through some things with my feelings...   
2  I'm going through some things with my feelings...   
3  I'm going through some things with my feelings...   
4  I'm going through some things with my feelings...   

                                            Response  
0  If everyone thinks you're worthless, then mayb...  
1  Hello, and thank you for your question and see...  
2  First thing I'd suggest is getting the sleep y...  
3  Therapy is essential for those that are feelin...  
4  I first want to let you know that you are not ...  

Cleaned and Annotated Data:
                                             Context  \
0  im going through some things with my feelings ...   
1  im going through some things with my feelings ...   
2  im going through some things with my feelings ...   
3  im going through some things with my feelings ...   
4  im goi

In [38]:
import os
from openai import OpenAI

# Correctly accessing the environment variable (assuming it's set outside this script)
api_key = os.environ.get('OPENAI_API_KEY')  # Make sure to set this in your environment variables

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)
# Function to generate a response from GPT-3.5 turbo
def get_response(prompt):
    response = client.Completion.create(
        engine="gpt-3.5-turbo",  # Specify the GPT-3.5 turbo model
        prompt=prompt,
        max_tokens=150,          # Adjust based on the average length of a response
        n=1,                     # Number of responses to generate
        stop=None,               # Any stopping criteria if necessary
        temperature=0.7          # Controls the randomness. Lower is more deterministic.
    )
    return response.choices[0].text.strip()

KeyError: 'OPENAI_API_KEY'

In [29]:
def chatbot_response(user_input):
    # Clean and prepare user input
    cleaned_input = clean_text(user_input)
    # Annotate input (optional, can be used for logging or conditional responses)
    annotation = annotate_data(cleaned_input)
    # Generate a response using GPT-3.5 turbo
    prompt = f"Mental health assistant: How can I assist you today?\nUser: {cleaned_input}\nMental health assistant:"
    response = get_response(prompt)
    return response


In [30]:
# Example of interacting with the chatbot
user_input = "I've been feeling really overwhelmed and stressed lately."
print("Chatbot Response:", chatbot_response(user_input))

APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [47]:
pip install -U langchain langchain-openai


   ---------------------------------------- 0.0/46.1 kB ? eta -:--:--
   -------- ------------------------------- 10.2/46.1 kB ? eta -:--:--
   -------- ------------------------------- 10.2/46.1 kB ? eta -:--:--
   ----------------- ---------------------- 20.5/46.1 kB 165.2 kB/s eta 0:00:01
   -------------------------- ------------- 30.7/46.1 kB 187.9 kB/s eta 0:00:01
   ---------------------------------------- 46.1/46.1 kB 209.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/799.0 kB ? eta -:--:--
   -- ------------------------------------- 41.0/799.0 kB 2.0 MB/s eta 0:00:01
   --- ----------------------------------- 71.7/799.0 kB 787.7 kB/s eta 0:00:01
   -------- ------------------------------- 174.1/799.0 kB 1.3 MB/s eta 0:00:01
   ------------- -------------------------- 266.2/799.0 kB 1.3 MB/s eta 0:00:01
   ---------------- ----------------------- 327.7/799.0 kB 1.2 MB/s eta 0:00:01
   ---------------------------- ----------- 563.2/799.0 kB 1.8 MB/s eta 0:00:

In [44]:
import streamlit as st
from langchain.chains import LLMChain
from langchain.schema import OpenAIConfig
from langchain.llms import OpenAI

# Setup
openai_config = OpenAIConfig(api_key="sk-None-dJfQwFpXzIxHqYtn4g5iT3BlbkFJkGGJlcbP7bYDM93Tdkd6")
llm = OpenAI(configuration=openai_config)
chain = LLMChain(llm)

def chatbot_response(prompt):
    response = chain.run(prompt)
    return response

# Streamlit interface
st.title("Mental Health Support Chatbot")
input_text = st.text_input("How can I assist you today?")
if input_text:
    response = chatbot_response(input_text)
    st.text_area("Response:", value=response, height=200, max_chars=None)

if __name__ == '__main__':
    st.run()

ImportError: cannot import name 'OpenAIConfig' from 'langchain.schema' (C:\Users\user\anaconda3\Lib\site-packages\langchain\schema\__init__.py)

In [51]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI

# Assuming OpenAI needs some form of API key setup, we'll configure it directly if no specific config class is provided
llm = OpenAI(api_key="sk-None-dJfQwFpXzIxHqYtn4g5iT3BlbkFJkGGJlcbP7bYDM93Tdkd6")

# Initialize the LLMChain with the llm instance
chain = LLMChain(llm=llm)

def chatbot_response(prompt):
    response = chain.run(prompt)
    return response

# Example usage
user_input = "I've been feeling really anxious lately, what should I do?"
print("Chatbot:", chatbot_response(user_input))

ValidationError: 1 validation error for LLMChain
prompt
  field required (type=value_error.missing)

In [52]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch
import pandas as pd

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-small')
model = GPT2LMHeadModel.from_pretrained('microsoft/DialoGPT-small')

# Load your dataset
df = pd.read_csv('Cleaned_Mental_Health_Conversational_Data.csv')
conversations = df['Context'].tolist() + df['Response'].tolist()

# Tokenize conversations
encodings = tokenizer(conversations, truncation=True, padding=True, max_length=512)

# Prepare dataset
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = ChatDataset(encodings)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

# Start training
trainer.train()

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [25]:
!pip install openai
!pip install datasets transformers[sentencepiece]
!pip install sentencepiece

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/547.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/547.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/547.8 kB ? eta -:--:--
    --------------------------------------- 10.2/547.8 kB ? eta -:--:--
    --------------------------------------- 10.2/547.8 kB ? eta -:--:--
    --------------------------------------- 10.2/547.8 kB ? eta -:--:--
    --------------------------------------- 10.2/547.8 kB ? eta -:--:--
    --------------------------------------- 10.2/547.8 kB ? eta -:--:--
   -- ------------------------------------- 30.7/547.8 kB 81.9 kB/s eta 0:00:07
   -- ------------------------------------- 30.7/547.8 kB 81.9 kB/s eta 0:00:07
   -- ------------------------------------- 30.7/547.8 kB 81.9 kB/s eta 0:00:07
   -- ------------------------------------- 30.7/547.8 kB 81

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.1 which is incompatible.



   -------------------- ------------------- 13.4/25.9 MB 47.8 kB/s eta 0:04:21
   -------------------- ------------------- 13.5/25.9 MB 48.1 kB/s eta 0:04:19
   -------------------- ------------------- 13.5/25.9 MB 48.1 kB/s eta 0:04:19
   -------------------- ------------------- 13.5/25.9 MB 48.1 kB/s eta 0:04:19
   -------------------- ------------------- 13.5/25.9 MB 48.1 kB/s eta 0:04:19
   -------------------- ------------------- 13.5/25.9 MB 48.1 kB/s eta 0:04:19
   -------------------- ------------------- 13.5/25.9 MB 48.1 kB/s eta 0:04:19
   -------------------- ------------------- 13.5/25.9 MB 48.1 kB/s eta 0:04:19
   -------------------- ------------------- 13.5/25.9 MB 48.1 kB/s eta 0:04:19
   -------------------- ------------------- 13.5/25.9 MB 48.1 kB/s eta 0:04:19
   -------------------- ------------------- 13.5/25.9 MB 48.1 kB/s eta 0:04:19
   -------------------- ------------------- 13.5/25.9 MB 47.8 kB/s eta 0:04:20
   -------------------- ------------------- 13.5/25

In [23]:
import os

# Temporary solution: set environment variable in the script
os.environ['OPENAI_API_KEY'] = 'sk-None-dJfQwFpXzIxHqYtn4g5iT3BlbkFJkGGJlcbP7bYDM93Tdkd6'

from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],
)

def chat_gpt4(prompt):
    response = client.completions.create(
        engine="text-davinci-003",  # Use the appropriate engine for GPT-4
        prompt=prompt,
        max_tokens=150
    )
    return response

# while True:
#     user_input = input("You: ")
#     response = chat_gpt4(user_input)
#     print(f"Bot: {response}")

In [26]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

def chat_t5(input_text):
    # Encode the input text
    inputs = tokenizer.encode("chatbot: " + input_text + " </s>", return_tensors="pt")
    # Generate the response
    outputs = model.generate(inputs, max_length=50, num_return_sequences=1)
    # Decode the response
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# while True:
#     user_input = input("You: ")
#     response = chat_t5(user_input)
#     print(f"Bot: {response}")

ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
